# Pergunta fundamental
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/<usuario>/<repo>/blob/<branch>/<caminho>/notebook.ipynb
)

Como transformar dados categóricos em algo que algoritmos entendam sem distorcer o significado?

Imagine que você é um chef de cozinha e recebe os ingredientes para um prato sofisticado. No entanto, os vegetais estão sujos de terra, algumas frutas estão amassadas e faltam temperos essenciais. Você iria simplesmente jogar tudo na panela? Claro que não. Você iria primeiro lavar, cortar, descartar o que não serve e organizar tudo.

No mundo dos dados, a realidade é muito parecida. Raramente recebemos um conjunto de dados ("dataset") perfeitamente limpo e pronto para uso. Na grande maioria das vezes, os dados brutos são "sujos": contêm valores ausentes, informações duplicadas, erros de digitação e formatos inconsistentes.

Nesta aula, vamos discutir como fazer exatamente esse "trabalho sujo" que, na verdade, é uma das etapas mais críticas e importantes de qualquer projeto de análise de dados. Vamos focar em quatro pilares da limpeza e preparação de dados:

* **Tratamento de Dados Ausentes**: O que fazer quando faltam informações?

* **Identificação e Remoção de Duplicatas**: Como garantir que cada registro seja único?

* **Análise de Outliers**: Como lidar com valores que fogem totalmente do padrão?

* **Conversão de Dados Categóricos**: Como transformar textos (categorias) em números que os algoritmos consigam entender?

Existe um ditado famoso da área: "Garbage in, garbage out" (Lixo entra, lixo sai). Nosso objetivo hoje é garantir que apenas dados de qualidade entrem em nossas análises.

# Bibliotecas necessárias

O pandas é a principal biblioteca em Python para manipulação e análise de dados. Ele foi desenvolvido para facilitar o trabalho com tabelas, algo que lembra muito o funcionamento de planilhas do Excel, mas com muito mais flexibilidade e integração com outras bibliotecas do ecossistema Python. A estrutura central do pandas é o DataFrame, que nada mais é do que uma tabela com linhas e colunas, onde cada coluna pode ter um tipo de dado diferente (números, texto, datas, valores booleanos, entre outros).

Enquanto o pandas é excelente para organizar, limpar e explorar os dados, muitas vezes precisamos dar um passo além: preparar esses dados para que sejam entendidos por algoritmos de aprendizado de máquina. É aí que entra o Scikit-Learn (sklearn), uma das bibliotecas mais populares para análise e aprendizado a partir de dados em Python. Dentro desse contexto, um desafio comum é lidar com variáveis categóricas, que aparecem como texto (nomes de cidades, cores, profissões etc.) mas precisam ser transformadas em números para que os algoritmos possam trabalhar com elas. Para isso, o sklearn oferece diferentes tipos de codificadores: **LabelEncoder, OneHotEnconder e OrdinalEncoder.**

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder

# Leitura do conjunto de dados


O pandas já oferece funções prontas para lidar com diferentes formatos. Isso significa que, independentemente de onde os dados estão armazenados, seja em um arquivo simples como CSV, TXT, XLSX ou até mesmo em dicionários criados diretamente no código, o pandas tem uma forma prática de transformá-los em um DataFrame, que é a estrutura principal de análise.

A tabela a seguir apresenta funções e os formatos de arquivos esperados.

| Função Pandas          | Formato de Entrada             | Exemplo de Uso                                                                 |
|------------------------|--------------------------------|---------------------------------------------------------------------------------|
| `pd.read_csv()`        | Arquivos CSV                  | `df = pd.read_csv("dados.csv")`                                                 |
| `pd.read_table()`      | Arquivos TXT (com separador)  | `df = pd.read_table("dados.txt", sep="\t")`                                     |
| `pd.read_excel()`      | Planilhas Excel (`.xlsx`, `.xls`) | `df = pd.read_excel("dados.xlsx", sheet_name="Plan1")`                          |
| `pd.read_json()`       | Arquivos JSON                 | `df = pd.read_json("dados.json")`                                               |
| `pd.read_sql()`        | Bancos de Dados (SQL)         | `df = pd.read_sql("SELECT * FROM tabela", con)`                                 |
| `pd.read_parquet()`    | Arquivos Parquet (otimizado)  | `df = pd.read_parquet("dados.parquet")`                                         |
| `pd.read_feather()`    | Arquivos Feather (otimizado)  | `df = pd.read_feather("dados.feather")`                                         |
| `pd.DataFrame()`       | Dados em memória (listas/dicionários) | `df = pd.DataFrame({"nome": ["Ana", "João"], "idade": [25, 30]})`   |

In [ ]:
# !ls drive/Shareddrives/AVD20252/Life Expectancy Data.csv
df = pd.read_csv("drive/Shareddrives/AVD20252/Life Expectancy Data.csv")

Podemos utilizar a estrutura do DataFrame para visualizar a estrutura dos dados lidos. No entanto, muitas vezes não precisamos visualizar tudo de uma vez, especialmente quando o arquivo tiver milhares de linhas. Para isso, usamos os métodos `.head()` e `.tail()`, que permitem inspecionar rapidamente apenas uma parte do DataFrame.

* `head():` mostra, por padrão, as 5 primeiras linhas do DataFrame e se torna bem útil para verificar se os dados foram lidos corretamente, observar o nome das colunas e ter uma noção inicial da estrutura. Também podemos passar um número como argumento, por exemplo `df.head(10)`, para ver as 10 primeiras linhas.

* `tail():` funciona da mesma forma, mas retorna as últimas linhas do DataFrame. Isso é especialmente útil quando queremos conferir se os dados no final da tabela foram importados corretamente, ou se há registros incompletos/estranhos no final do arquivo. Exemplo: `df.tail(3)` retorna as 3 últimas linhas.

In [ ]:
df[['Country', 'Population']].head(3)

,Country,Population
0,Afghanistan,33736494.0
1,Afghanistan,327582.0
2,Afghanistan,31731688.0


## Verificando se os dados fazem sentido

Quando carregamos um conjunto de dados no pandas, duas formas muito úteis de compreender sua estrutura são os métodos `.shape` e `.info()`. O atributo shape é simples e direto: ele retorna uma tupla que indica o número de linhas e colunas do DataFrame. Em outras palavras, é a maneira mais rápida de responder "quantos registros eu tenho?" e "quantas variáveis estão sendo analisadas?". Por exemplo, se o resultado for (1000, 12), isso significa que temos mil linhas (observações) distribuídas em doze colunas (atributos). Esse tipo de informação é fundamental logo no início, pois nos dá uma ideia do tamanho do problema que estamos lidando e até mesmo do esforço computacional que pode ser necessário.

Já o método `info()` fornece uma visão mais detalhada. Ele apresenta um resumo sobre a estrutura do DataFrame, listando todas as colunas, seus respectivos tipos de dados e a quantidade de valores não nulos em cada uma delas. Esse detalhe é especialmente importante, pois permite identificar rapidamente a presença de valores ausentes, além de verificar se os tipos de dados estão coerentes com o que esperamos. Por exemplo, se a coluna "idade" aparece como object, isso pode indicar que há valores não numéricos ou registros incorretos que precisam ser tratados. Além disso, o `info()` mostra também a memória utilizada, o que pode ser relevante quando trabalhamos com bases muito grandes.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2938 non-null   object 
 1   Year                             2938 non-null   int64  
 2   Status                           2938 non-null   object 
 3   Life expectancy                  2928 non-null   float64
 4   Adult Mortality                  2928 non-null   float64
 5   infant deaths                    2938 non-null   int64  
 6   Alcohol                          2744 non-null   float64
 7   percentage expenditure           2938 non-null   float64
 8   Hepatitis B                      2385 non-null   float64
 9   Measles                          2938 non-null   int64  
 10   BMI                             2904 non-null   float64
 11  under-five deaths                2938 non-null   int64  
 12  Polio               

(2938, 22)

# Tratando dados ausentes

um passo natural é verificar se existem valores ausentes. Em pandas, o método mais utilizado para isso é o `isnull()`. Ele retorna, para cada célula do DataFrame, um valor booleano: True se o dado for nulo (ausente) e False caso contrário.

Na prática, dificilmente olhamos para esse resultado "cru", pois seria apenas uma tabela de True e False. O uso mais comum é combiná-lo com o método `sum()`. Dessa forma, em vez de verificar célula por célula, conseguimos contar quantos valores nulos existem em cada coluna. Por exemplo, ao usar `df.isnull().sum()`, o pandas retorna uma série onde o índice são os nomes das colunas e os valores indicam quantos registros ausentes cada uma possui. Isso nos dá uma visão clara de onde estão os problemas de completude no conjunto de dados.

In [ ]:
df.isnull().sum()

,0
Country,0
Year,0
Status,0
Life expectancy,10
Adult Mortality,10
infant deaths,0
Alcohol,194
percentage expenditure,0
Hepatitis B,553
Measles,0


Quando aplicamos a expressão `df.isnull().sum() / df.shape[0] * 100`, o que obtemos é a porcentagem de valores ausentes em cada coluna do DataFrame. O processo funciona assim: primeiro, `df.isnull()` cria uma matriz booleana indicando se cada célula é nula (`True`) ou não (`False`). Em seguida, o `.sum()` soma os valores True de cada coluna, resultando no total de valores ausentes por coluna. Esse número é então dividido pelo total de linhas do DataFrame (`df.shape[0]`), o que gera a fração de dados faltantes. Por fim, multiplicamos por 100 para converter essa fração em porcentagem.

Isso é muito útil porque nos dá uma visão quantitativa clara sobre a qualidade dos dados. Por exemplo, se apenas 1% de uma coluna está ausente, talvez possamos simplesmente ignorar ou preencher esses poucos valores. Mas se 70% de uma coluna estiver faltando, pode ser melhor remover essa variável da análise, já que ela não acrescenta informação confiável.

In [ ]:
df.isnull().sum() / df.shape[0] * 100

,0
Country,0.000000
Year,0.000000
Status,0.000000
Life expectancy,0.340368
Adult Mortality,0.340368
infant deaths,0.000000
Alcohol,6.603131
percentage expenditure,0.000000
Hepatitis B,18.822328
Measles,0.000000


# Tratando dados duplicados

O método `df.duplicated()` retorna uma série booleana indicando, para cada linha do DataFrame, se ela é uma duplicata (`True`) ou não (`False`). A checagem é feita considerando todas as colunas, a menos que especifiquemos um subconjunto.

Normalmente, em vez de olhar linha a linha, usamos esse método em conjunto com o `.sum()` para contar quantos registros duplicados existem. Assim, `df.duplicated().sum()` nos diz diretamente o número total de linhas repetidas. Isso é importante porque dados duplicados podem distorcer análises estatísticas, enviesar médias e até prejudicar modelos de aprendizado de máquina, dando a falsa impressão de haver mais ocorrências de certos casos do que realmente existem.

Podemos também especificar apenas algumas colunas para verificar duplicidade. Por exemplo, `df.duplicated(subset=["id"])` identifica se há registros repetidos com base apenas na coluna id. Esse uso é comum quando queremos garantir unicidade de chaves primárias, como CPF, matrícula ou identificador de cliente.

Caso seja necessário remover duplicatas, basta combinar com o método `drop_duplicates()`. Por exemplo, `df.drop_duplicates(inplace=True)` elimina diretamente todas as linhas duplicadas do DataFrame.

In [ ]:
df2 = df.drop(columns=['Alcohol'])
df2
df

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,Zimbabwe,2004,Developing,44.3,723.0,27,4.36,0.000000,68.0,31,...,67.0,7.13,65.0,33.6,454.366654,12777511.0,9.4,9.4,0.407,9.2
2934,Zimbabwe,2003,Developing,44.5,715.0,26,4.06,0.000000,7.0,998,...,7.0,6.52,68.0,36.7,453.351155,12633897.0,9.8,9.9,0.418,9.5
2935,Zimbabwe,2002,Developing,44.8,73.0,25,4.43,0.000000,73.0,304,...,73.0,6.53,71.0,39.8,57.348340,125525.0,1.2,1.3,0.427,10.0
2936,Zimbabwe,2001,Developing,45.3,686.0,25,1.72,0.000000,76.0,529,...,76.0,6.16,75.0,42.1,548.587312,12366165.0,1.6,1.7,0.427,9.8


# Tratando outliers

Se quisermos identificar valores que fogem muito do padrão dos dados, podemos recorrer ao conceito de outliers. Eles representam pontos que estão muito distantes da maior parte das observações e podem distorcer estatísticas ou análises. Uma forma comum de detectá-los é usar o método do intervalo interquartílico (IQR). Nesse caso, calculamos o primeiro quartil (Q1) e o terceiro quartil (Q3), e definimos o IQR como a diferença entre eles. Valores abaixo de `Q1 - 1.5*IQR` ou acima de `Q3 + 1.5*IQR` são considerados outliers.

Então, se rodarmos o cálculo `df[(df["coluna"] < Q1 - 1.5*IQR) | (df["coluna"] > Q3 + 1.5*IQR)]`, teremos um subconjunto do DataFrame contendo apenas as linhas que foram classificadas como outliers de acordo com essa regra.

Outra forma de identificar outliers é pelo desvio padrão. Nesse caso, assumimos que a maior parte dos dados deve estar próxima da média, e valores que se afastam demais podem ser considerados extremos. Uma regra prática bastante usada é considerar como outliers aqueles pontos que estão a mais de 3 desvios padrão da média.

Então, se rodarmos o cálculo `df[(df["coluna"] < media - 3*desvio) | (df["coluna"] > media + 3*desvio)]`, teremos um subconjunto do DataFrame contendo apenas as linhas que estão muito abaixo ou muito acima do valor médio esperado, de acordo com esse critério estatístico.

In [ ]:
mean = df.Alcohol.mean()
std = df.Alcohol.std()
print(mean, std)

df[(df.Alcohol < mean - 3*std)| (df.Alcohol > mean + 3*std)]

4.602860787172012 4.052412658755652


,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
228,Belarus,2011,Developing,72.0,232.0,0,17.31,846.911307,98.0,50,...,98.0,4.92,98.0,0.1,6519.71753,9473172.0,2.0,2.1,0.787,15.5
873,Estonia,2008,Developing,74.2,167.0,0,16.99,225.072362,94.0,0,...,95.0,6.60,95.0,0.1,1894.54850,13379.0,2.0,2.1,0.835,16.1
874,Estonia,2007,Developing,73.0,189.0,0,17.87,1904.124690,95.0,1,...,95.0,5.16,95.0,0.1,16586.45200,13468.0,2.0,2.1,0.829,16.1


# Tratando lixo

Quando usamos `df[i].value_counts()`, estamos pedindo ao pandas para contar quantas vezes cada valor aparece na coluna i. O resultado é uma série ordenada, onde o índice são os valores únicos encontrados naquela coluna e os números correspondem às suas frequências absolutas.

Isso é muito útil em variáveis categóricas ou discretas, porque rapidamente conseguimos ver quais categorias são mais comuns, se há valores raros demais, ou até se existem valores estranhos (como “?” ou “-”) que foram lidos como categorias. Também pode ser aplicado em colunas numéricas discretas para verificar distribuição de valores.

Por exemplo, considere o DataFrame:

| id | cor      |
|----|----------|
| 1  | azul     |
| 2  | vermelho |
| 3  | azul     |
| 4  | verde    |
| 5  | azul     |

Se aplicarmos `df["cor"].value_counts()`, teremos:

```shell
azul       3
vermelho   1
verde      1
Name: cor, dtype: int64
```


Se quisermos ainda selecionar colunas com base no tipo de dado, o método `select_dtypes()` permite filtrar apenas as colunas de certos tipos (por exemplo, apenas numéricas, apenas categóricas, apenas booleanas etc.). Quando adicionamos `.columns` no final, obtemos apenas os nomes dessas colunas.

Então, se rodarmos `df.select_dtypes(include="number").columns` teremos uma lista com os nomes das colunas que são do tipo number.

In [ ]:
df.select_dtypes(include="number").columns
df.Status.value_counts()
df.dropna()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,Zimbabwe,2004,Developing,44.3,723.0,27,4.36,0.000000,68.0,31,...,67.0,7.13,65.0,33.6,454.366654,12777511.0,9.4,9.4,0.407,9.2
2934,Zimbabwe,2003,Developing,44.5,715.0,26,4.06,0.000000,7.0,998,...,7.0,6.52,68.0,36.7,453.351155,12633897.0,9.8,9.9,0.418,9.5
2935,Zimbabwe,2002,Developing,44.8,73.0,25,4.43,0.000000,73.0,304,...,73.0,6.53,71.0,39.8,57.348340,125525.0,1.2,1.3,0.427,10.0
2936,Zimbabwe,2001,Developing,45.3,686.0,25,1.72,0.000000,76.0,529,...,76.0,6.16,75.0,42.1,548.587312,12366165.0,1.6,1.7,0.427,9.8


## Tratando dados faltantes

Quando trabalhamos com análise de dados, quase sempre nos deparamos com o problema de dados faltantes. Eles podem surgir por diversos motivos: erros na coleta, falhas no sistema, informações que não se aplicam a todos os casos ou simplesmente omissões. O ponto central é que valores ausentes podem comprometer a qualidade da análise, enviesar resultados e até inviabilizar o uso de modelos de machine learning.

O primeiro passo é detectar esses valores. Em pandas, já vimos que métodos como `df.isnull().sum()` ou a porcentagem de nulos ajudam a quantificar o problema. A partir daí, precisamos escolher a melhor estratégia de tratamento, e essa escolha depende tanto da quantidade de dados ausentes quanto da importância da variável no contexto da análise.

Em casos mais simples, podemos remover registros. Se uma coluna tiver poucos valores nulos (por exemplo, menos de 5%), eliminar essas linhas (`df.dropna()`) pode ser aceitável. Da mesma forma, se uma coluna inteira tiver uma quantidade enorme de ausências (digamos, mais de 70%), pode ser mais eficiente descartar a coluna (`df.drop(columns=["coluna"])`) em vez de tentar recuperar informações quase inexistentes.

Outra abordagem comum é a **imputação**, que consiste em substituir os valores faltantes por estimativas. Para variáveis numéricas, podemos preencher com a **média**, **mediana** ou **moda**, dependendo do que melhor representa a distribuição dos dados. Já para variáveis categóricas, a **moda** é uma escolha frequente, pois mantém a coerência das categorias.

Para preencher uma coluna com dado faltante considerando a mediana, podemos usar o comando `df[i].fillna(df[i].median(), inplace=True)`, em que i define qual a coluna será preenchida.

Importante! A mediana é uma medida de tendência central que não é influenciada por valores extremos (outliers). Por isso, é muitas vezes preferida à média quando a distribuição da variável é assimétrica ou contém valores anômalos. Por exemplo, em uma coluna de salário, se a maioria ganha entre 2 e 5 mil mas existe alguém com 100 mil, a média vai ser puxada para cima, enquanto a mediana continua representando melhor o "valor típico".

O parâmetro inplace define se a operação feita no DataFrame deve alterar o objeto original diretamente ou se deve retornar uma cópia modificada. Ou seja, os valores faltantes de `df[i]` são sobrescritos pela mediana. Caso usássemos `inplace=False` (padrão), teríamos que salvar o resultado em uma nova variável ou reatribuir a coluna.


In [ ]:
df.Alcohol.fillna(df.Alcohol.median(), inplace=True)
df.isnull().sum()

/tmp/ipython-input-3279755289.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.Alcohol.fillna(df.Alcohol.median(), inplace=True)


,0
Country,0
Year,0
Status,0
Life expectancy,10
Adult Mortality,10
infant deaths,0
Alcohol,0
percentage expenditure,0
Hepatitis B,553
Measles,0


# Tratando dados não numéricos

## Label Encoding

O **Label Encoding** (codificação de rótulos) consiste em transformar cada categoria de uma variável categórica em um valor numérico único. Em outras palavras, atribui-se um número inteiro para cada categoria distinta.

Por exemplo, poderíamos codificar as categorias de cor {vermelho, verde, azul} como {0, 1, 2} respectivamente, onde cada número é apenas um rótulo sem significado numérico real além de identificar a categoria. Essa técnica é simples e mantém o conjunto de dados compacto (apenas uma nova coluna numérica é criada para representar a categoria). Muitas bibliotecas, como o Scikit-Learn, oferecem implementações prontas (como por exemplo, a classe LabelEncoder).

| ID | cor      | cor_label |
|----|----------|-----------|
| 1  | vermelho | 0         |
| 2  | verde    | 1         |
| 3  | azul     | 2         |
| 4  | verde    | 1         |
| 5  | vermelho | 0         |

No entanto, é importante notar uma característica do **Label Encoding**: ele impõe uma ordem. Como os rótulos são números, o modelo pode interpretar que existe uma hierarquia ou relação de maior/menor entre eles, mesmo quando as categorias originais não possuem ordem intrínseca.

Por exemplo, se atribuirmos 1 a Red (vermelho), 2 a Green (verde) e 3 a Blue (azul), isso pode levar o algoritmo a supor que verde é “maior” que vermelho e azul é maior que ambos, uma relação que não faz sentido na realidade. Esse comportamento pode prejudicar o desempenho do modelo, pois ele pode inferir padrões onde não existem. Logo, Label Encoding é mais adequado quando as categorias possuem uma ordem natural (dados ordinais), caso contrário, deve-se ter cautela, pois os valores numéricos podem sugerir uma ordem falsa entre categorias não ordenadas.

Para uso, aplica-se o método fit_transform: `df.fit_transform(df[["column"]])`

In [ ]:
le = LabelEncoder()
df['Status_label'] = le.fit_transform(df['Status'])
df

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Status_label
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1,1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,1
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9,1
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8,1
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,Zimbabwe,2004,Developing,44.3,723.0,27,4.36,0.000000,68.0,31,...,7.13,65.0,33.6,454.366654,12777511.0,9.4,9.4,0.407,9.2,1
2934,Zimbabwe,2003,Developing,44.5,715.0,26,4.06,0.000000,7.0,998,...,6.52,68.0,36.7,453.351155,12633897.0,9.8,9.9,0.418,9.5,1
2935,Zimbabwe,2002,Developing,44.8,73.0,25,4.43,0.000000,73.0,304,...,6.53,71.0,39.8,57.348340,125525.0,1.2,1.3,0.427,10.0,1
2936,Zimbabwe,2001,Developing,45.3,686.0,25,1.72,0.000000,76.0,529,...,6.16,75.0,42.1,548.587312,12366165.0,1.6,1.7,0.427,9.8,1


## One-Hot Encoding

O **One-Hot Encoding** é uma técnica que converte cada categoria em uma nova variável binária (chamada variável dummy). Em vez de usar um único número para representar todas as categorias de uma coluna, o one-hot cria colunas adicionais, uma para cada categoria única, preenchendo-as com 1 ou 0 conforme a presença da categoria naquela instância.

Voltando ao exemplo das cores {vermelho, verde, azul}, o **One-Hot Encoding** criaria três colunas distintas, digamos, cor_vermelho, cor_verde e cor_azul. Se um registro tem valor “verde” nessa variável, então cor_verde = 1 e nas outras duas colunas seria 0. Assim, cada exemplo é representado por um vetor binário indicando qual categoria está presente.

| ID | cor      | cor_vermelho | cor_verde | cor_azul |
|----|----------|--------------|-----------|----------|
| 1  | vermelho | 1            | 0         | 0        |
| 2  | verde    | 0            | 1         | 0        |
| 3  | azul     | 0            | 0         | 1        |
| 4  | verde    | 0            | 1         | 0        |
| 5  | vermelho | 1            | 0         | 0        |

A grande vantagem do One-Hot Encoding é não introduzir nenhuma ordenação artificial. Cada categoria é tratada de forma independente, como uma dimensão própria, evitando que o modelo interprete alguma hierarquia entre elas. Desse modo, resolvemos o problema mencionado no Label Encoding: no one-hot, azul, verde e vermelho serão codificados em colunas separadas, e o modelo "entende" que são apenas categorias diferentes, sem relação de maior ou menor entre si.

Essa técnica é muito eficaz para variáveis nominais (categorias sem ordem inerente), garantindo que o algoritmo não assuma ordens inexistentes.

## Ordinal Encoding

O **Ordinal Encoding** é indicado quando as categorias possuem **ordem natural** (hierarquia), como nível de escolaridade, tamanho de roupa ou faixas de satisfação. Diferente do Label Encoding (que atribui números arbitrários) e do One-Hot (que cria colunas binárias independentes), o ordinal preserva a **ordem intrínseca** entre as classes.

Exemplo com escolaridade: {fundamental < médio < superior < mestrado < doutorado}.

| ID | escolaridade | escolaridade_ordinal |
|----|--------------|----------------------|
| 1  | fundamental  | 0                    |
| 2  | médio        | 1                    |
| 3  | superior     | 2                    |
| 4  | mestrado     | 3                    |
| 5  | doutorado    | 4                    |